<a href="https://colab.research.google.com/github/hamzaskhan/yolov8-catch-pixel-coordinates-of-detected-objects/blob/main/yolov8_objectDetection_with_PixelCoordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is a confirmation code to visualise the bounding boxes

# Step 1: Install the required library
!pip install ultralytics

# Step 2: Import necessary libraries
from ultralytics import YOLO
from google.colab import files
import zipfile
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Step 3: Verify YOLO installation
model = YOLO('yolov8n.pt')  # Load a pre-trained YOLOv8 nano model
print("YOLOv8 successfully loaded!")

# Step 4: Upload and extract the dataset
print("Upload the dataset zip file:")
uploaded = files.upload()  # This will prompt you to upload your dataset ZIP file

# Extract the uploaded zip file
zip_file_name = list(uploaded.keys())[0]  # Get the uploaded ZIP file name
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("/content")  # Extract to the /content directory
print("Dataset extracted successfully!")

# Step 5: Create the dataset configuration YAML file
with open("map_dataset.yaml", "w") as f:
    f.write("""
    path: /content
    train: train/images
    val: valid/images
    test: test/images
    nc: 2
    names: ['Mountain', 'Tree']
    """)
print("YAML configuration file created!")

# Step 6: Train the YOLO model
print("Training the YOLO model...")
model.train(
    data='map_dataset.yaml',  # Path to your dataset YAML file
    epochs=15,                # Number of training epochs
    imgsz=640,                # Image size
    batch=8,                  # Batch size
    name='yolo_map_detection' # Name for this training run
)
print("Model training completed!")

# Step 7: Validate the model
print("Validating the YOLO model...")
metrics = model.val(data='map_dataset.yaml')
print("Validation metrics:", metrics)

# Step 8: Load the trained model
print("Loading the trained YOLO model...")
model = YOLO('/content/runs/detect/yolo_map_detection/weights/best.pt')
print("Trained model loaded!")

# Step 9: Test the model on a specific image
print("Upload a test image:")
uploaded = files.upload()  # Prompt to upload a test image
test_image_path = list(uploaded.keys())[0]  # Get the test image file name

# Run inference on the uploaded image
results = model.predict(source=test_image_path, conf=0.5, save=False)  # Confidence threshold: 50%

# Step 10: Extract and display detection results
detections = results[0].boxes  # Get detected boxes
print("\nDetected Objects and Their Locations:")

# Open the image for drawing
img = Image.open(test_image_path)
draw = ImageDraw.Draw(img)

for box in detections:
    cls = int(box.cls[0])  # Class index
    conf = float(box.conf[0])  # Confidence score
    x1, y1, x2, y2 = box.xyxy[0].tolist()  # Bounding box coordinates (top-left and bottom-right)
    label = results[0].names[cls]  # Class name

    # Print detection details
    print(f"Class: {label}, Confidence: {conf:.2f}, Location: x1={x1:.2f}, y1={y1:.2f}, x2={x2:.2f}, y2={y2:.2f}")

    # Draw bounding box on the image
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)  # Red box
    draw.text((x1, y1 - 10), f"{label} {conf:.2f}", fill="red")  # Label above the box

# Step 11: Visualize the image with bounding boxes
plt.imshow(img)
plt.axis('off')
plt.show()

# Annotated image is displayed above
